In [ ]:
pip install torch==2.5.0 torchvision==0.20.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.5/906.5 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 90.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 86.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 59.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 71.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20

In [ ]:
#image testing code
import cv2
import numpy as np
import torch
import smtplib
import pandas as pd
from email.message import EmailMessage
from geopy.distance import geodesic
import matplotlib.pyplot as plt
from google.colab import drive, files

# 🔹 Mount Google Drive
drive.mount('/content/drive')

# 🔥 Load YOLOv9 Model (Fire + Smoke Detection)
MODEL_PATH = "/content/drive/MyDrive/yolov9/SAVED_BEST.pt"
model = torch.hub.load('WongKinYiu/yolov9', 'custom', path=MODEL_PATH, source='github', force_reload=True)

# 🔥 Load Fire Station Data
fire_stations_df = pd.read_csv("/content/drive/MyDrive/yolov9/coimbatore_region_fire_stations.csv")
fire_stations_df.rename(columns={"Latitude": "latitude", "Longitude": "longitude"}, inplace=True)

# 🔥 Fixed Fire Location (KIT College)
fire_latitude, fire_longitude = 10.9993751, 77.0843437
maps_link = f"https://www.google.com/maps?q={fire_latitude},{fire_longitude}"

# 🔹 Function to Find Nearest Fire Station
def find_nearest_fire_station(fire_lat, fire_lon):
    min_distance = float("inf")
    nearest_station = None
    for _, row in fire_stations_df.iterrows():
        station_coords = (row["latitude"], row["longitude"])
        fire_coords = (fire_lat, fire_lon)
        distance = geodesic(station_coords, fire_coords).kilometers
        if distance < min_distance:
            min_distance = distance
            nearest_station = row
    return nearest_station, min_distance

# 🔹 Email Credentials (Store password securely instead of hardcoding)
SENDER_EMAIL = "yolov9project@gmail.com"
APP_PASSWORD = "yfus foac dwcu wrgm"  # ⚠ Replace with a secure method (e.g., environment variable)

# 🔹 Function to Send Email Alert
def send_email_alert(recipient_email, subject, body):
    if not recipient_email:
        print("❌ No email address found for the nearest fire station!")
        return
    msg = EmailMessage()
    msg["Subject"] = subject
    msg["From"] = SENDER_EMAIL
    msg["To"] = recipient_email
    msg.set_content(body)
    try:
        with smtplib.SMTP_SSL("smtp.gmail.com", 465) as server:
            server.login(SENDER_EMAIL, APP_PASSWORD)
            server.send_message(msg)
        print(f"✅ Email alert sent successfully to {recipient_email}!")
    except Exception as e:
        print(f"❌ Failed to send email: {e}")

# 🔹 Function to Process Uploaded Image
def process_uploaded_image(image_path):
    # 🔹 Read image
    frame = cv2.imread(image_path)
    if frame is None:
        print("❌ Error: Unable to load image.")
        return

    # 🔹 Display original image
    plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

    # 🔥 Fire & Smoke Detection
    results = model(image_path)  # Use path, not frame
    results_df = results.pandas().xyxy[0]  # Convert to Pandas DataFrame (xyxy format)

    # 🔥 Detect Fire and Smoke
    fire_detected = results_df[(results_df["name"] == "fire") & (results_df["confidence"] > 0.05)]
    smoke_detected = results_df[(results_df["name"] == "smoke") & (results_df["confidence"] > 0.05)]

    frame_copy = frame.copy()  # Ensure mutable image

    fire_alert = not fire_detected.empty
    smoke_alert = not smoke_detected.empty

    # 🔹 Draw bounding boxes if fire or smoke is detected
    if fire_alert:
        print("🔥 Fire detected!")
        for _, row in fire_detected.iterrows():
            x1, y1, x2, y2 = int(row["xmin"]), int(row["ymin"]), int(row["xmax"]), int(row["ymax"])
            cv2.rectangle(frame_copy, (x1, y1), (x2, y2), (0, 0, 255), 2)  # Red for fire

    if smoke_alert:
        print("💨 Smoke detected!")
        for _, row in smoke_detected.iterrows():
            x1, y1, x2, y2 = int(row["xmin"]), int(row["ymin"]), int(row["xmax"]), int(row["ymax"])
            cv2.rectangle(frame_copy, (x1, y1), (x2, y2), (200, 200, 200), 2)  # Grey for smoke

    # 🔹 Display Detection Results
    plt.imshow(cv2.cvtColor(frame_copy, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

    # 🔥 If fire or smoke is detected, find the nearest fire station
    if fire_alert or smoke_alert:
        nearest_fire_station, distance_to_station = find_nearest_fire_station(fire_latitude, fire_longitude)

        if nearest_fire_station is not None:
            fire_station_info = f"{nearest_fire_station['Station_Name']} (📍 {nearest_fire_station['latitude']}, {nearest_fire_station['longitude']})"
            fire_station_email = nearest_fire_station.get("email", "").strip()

            # 🔥🔥 Prepare Email Alert Body
            alert_types = []
            if fire_alert:
                alert_types.append("🔥 Fire")
            if smoke_alert:
                alert_types.append("💨 Smoke")

            alert_summary = " and ".join(alert_types)
            email_body = (f"{alert_summary} detected at KIT College!\n\n"
                          f"📍 Location: {fire_latitude}, {fire_longitude}\n"
                          f"🌍 View on Map: {maps_link}\n"
                          f"🚒 Nearest Fire Station: {fire_station_info}\n"
                          f"📏 Distance: {distance_to_station:.2f} km\n\n"
                          "Immediate action required!")

            # 🔹 Send Combined Email Alert
            send_email_alert(fire_station_email, f"{alert_summary} Alert!", email_body)

            print(f"🚒 Nearest Fire Station: {fire_station_info} (📧 {fire_station_email})")
            print(f"📏 Distance: {distance_to_station:.2f} km")
    else:
        print("✅ No fire or smoke detected!")

# 🔹 Upload Image and Run Detection
uploaded = files.upload()
for filename in uploaded.keys():
    image_path = f"/content/{filename}"  # Save the uploaded file
    process_uploaded_image(image_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Downloading: "https://github.com/WongKinYiu/yolov9/zipball/main" to /root/.cache/torch/hub/main.zip
YOLO 🚀 2025-4-24 Python-3.11.12 torch-2.5.0+cu124 CUDA:0 (Tesla T4, 15095MiB)

/root/.cache/torch/hub/WongKinYiu_yolov9_main/models/experimental.py:243: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don'

Saving f1.jpg to f1 (1).jpg
🔥 Fire detected!


/root/.cache/torch/hub/WongKinYiu_yolov9_main/models/common.py:1063: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


✅ Email alert sent successfully to kit.25.21bad040@gmail.com!
🚒 Nearest Fire Station: Sulur Fire Station (📍 11.0242, 77.127) (📧 kit.25.21bad040@gmail.com)
📏 Distance: 5.41 km


In [ ]:
# video testing code
import cv2
import numpy as np
import torch
import smtplib
import pandas as pd
from email.message import EmailMessage
from geopy.distance import geodesic
import matplotlib.pyplot as plt
from google.colab import drive, files
from IPython.display import clear_output
import time
import os

# 🔹 Mount Google Drive
drive.mount('/content/drive')

# 🔥 Load YOLOv9 Fire and Smoke Detection Model
MODEL_PATH = "/content/drive/MyDrive/yolov9/SAVED_BEST.pt"
model = torch.hub.load('WongKinYiu/yolov9', 'custom', path=MODEL_PATH, source='github', force_reload=True)

# 🔥 Load Fire Station Data
fire_stations_df = pd.read_csv("/content/drive/MyDrive/yolov9/coimbatore_region_fire_stations.csv")
fire_stations_df.rename(columns={"Latitude": "latitude", "Longitude": "longitude"}, inplace=True)

# 🔥 Fixed Fire Location (KIT College)
fire_latitude, fire_longitude = 10.9993751, 77.0843437
maps_link = f"https://www.google.com/maps?q={fire_latitude},{fire_longitude}"

# 🔹 Function to Find Nearest Fire Station
def find_nearest_fire_station(fire_lat, fire_lon):
    min_distance = float("inf")
    nearest_station = None
    for _, row in fire_stations_df.iterrows():
        station_coords = (row["latitude"], row["longitude"])
        fire_coords = (fire_lat, fire_lon)
        distance = geodesic(station_coords, fire_coords).kilometers
        if distance < min_distance:
            min_distance = distance
            nearest_station = row
    return nearest_station, min_distance

# 🔹 Email Credentials
SENDER_EMAIL = "yolov9project@gmail.com"
APP_PASSWORD = "yfus foac dwcu wrgm"

# 🔹 Function to Send Email Alert
def send_email_alert(recipient_email, subject, body):
    if not recipient_email:
        print("❌ No email address found for the nearest fire station!")
        return
    msg = EmailMessage()
    msg["Subject"] = subject
    msg["From"] = SENDER_EMAIL
    msg["To"] = recipient_email
    msg.set_content(body)
    try:
        with smtplib.SMTP_SSL("smtp.gmail.com", 465) as server:
            server.login(SENDER_EMAIL, APP_PASSWORD)
            server.send_message(msg)
        print(f"✅ Email alert sent successfully to {recipient_email}!")
    except Exception as e:
        print(f"❌ Failed to send email: {e}")

# 🔹 Function to Process Video in Real-Time
def process_video(video_path):
    cap = cv2.VideoCapture(video_path)

    fire_active = False
    smoke_active = False
    alert_sent_fire = False
    alert_sent_smoke = False

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # 🔥 Fire & Smoke Detection
        results = model(frame)
        results_df = results.pandas().xyxy[0]

        # 🔥 Fire Detection
        fire_boxes = results_df[(results_df["name"] == "fire") & (results_df["confidence"] > 0.05)]

        # 💨 Smoke Detection
        smoke_boxes = results_df[(results_df["name"] == "smoke") & (results_df["confidence"] > 0.05)]

        # 🔥 Handle Fire Detection
        if not fire_boxes.empty:
            if not fire_active:
                print("🔥 Fire detected! Alerting fire station...")
                fire_active = True
                alert_sent_fire = False

            for _, row in fire_boxes.iterrows():
                x1, y1, x2, y2 = int(row["xmin"]), int(row["ymin"]), int(row["xmax"]), int(row["ymax"])
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)  # Red for fire

            if not alert_sent_fire:
                nearest_fire_station, distance_to_station = find_nearest_fire_station(fire_latitude, fire_longitude)

                if nearest_fire_station is not None:
                    fire_station_info = f"{nearest_fire_station['Station_Name']} (📍 {nearest_fire_station['latitude']}, {nearest_fire_station['longitude']})"
                    fire_station_email = nearest_fire_station.get("email", "").strip()
                    email_body_fire = (f"🔥 Fire detected at KIT College!\n\n"
                                       f"📍 Location: {fire_latitude}, {fire_longitude}\n"
                                       f"🌍 View on Map: {maps_link}\n"
                                       f"🚒 Nearest Fire Station: {fire_station_info}\n"
                                       f"📏 Distance: {distance_to_station:.2f} km\n\n"
                                       "Immediate action required!")
                    send_email_alert(fire_station_email, "🔥 Fire Alert!", email_body_fire)

                    print(f"🚒 Nearest Fire Station: {fire_station_info} (📧 {fire_station_email})")
                    print(f"📏 Distance: {distance_to_station:.2f} km")

                    alert_sent_fire = True

        else:
            if fire_active:
                print("✅ Fire extinguished! Monitoring again...")
                fire_active = False
                alert_sent_fire = False

        # 💨 Handle Smoke Detection
        if not smoke_boxes.empty:
            if not smoke_active:
                print("💨 Smoke detected! Sending alert...")
                smoke_active = True
                alert_sent_smoke = False

            for _, row in smoke_boxes.iterrows():
                x1, y1, x2, y2 = int(row["xmin"]), int(row["ymin"]), int(row["xmax"]), int(row["ymax"])
                cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 255, 0), 2)  # Yellow for smoke

            if not alert_sent_smoke:
                nearest_fire_station, distance_to_station = find_nearest_fire_station(fire_latitude, fire_longitude)

                if nearest_fire_station is not None:
                    fire_station_info = f"{nearest_fire_station['Station_Name']} (📍 {nearest_fire_station['latitude']}, {nearest_fire_station['longitude']})"
                    fire_station_email = nearest_fire_station.get("email", "").strip()
                    email_body_smoke = (f"💨 Smoke detected at KIT College!\n\n"
                                        f"📍 Location: {fire_latitude}, {fire_longitude}\n"
                                        f"🌍 View on Map: {maps_link}\n"
                                        f"🚒 Nearest Fire Station: {fire_station_info}\n"
                                        f"📏 Distance: {distance_to_station:.2f} km\n\n"
                                        "Immediate action required!")
                    send_email_alert(fire_station_email, "💨 Smoke Alert!", email_body_smoke)

                    print(f"🚒 Nearest Fire Station: {fire_station_info} (📧 {fire_station_email})")
                    print(f"📏 Distance: {distance_to_station:.2f} km")

                    alert_sent_smoke = True

        else:
            if smoke_active:
                print("✅ Smoke cleared! Monitoring again...")
                smoke_active = False
                alert_sent_smoke = False

        # 🔹 Display Frame
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        clear_output(wait=True)
        plt.imshow(frame_rgb)
        plt.axis("off")
        plt.show()

    cap.release()
    print("✅ Video processing completed.")

# 🔹 Upload Video Manually
print("📥 Please upload your video file...")
uploaded_files = files.upload()

# 🔹 Process Uploaded Videos
for video_name in uploaded_files.keys():
    video_path = f"/content/{video_name}"
    print(f"📥 Processing: {video_path}")
    process_video(video_path)

    # ✅ Clean up after processing
    os.remove(video_path)
    print(f"🗑 Deleted processed file: {video_name}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Downloading: "https://github.com/WongKinYiu/yolov9/zipball/main" to /root/.cache/torch/hub/main.zip
YOLO 🚀 2025-4-24 Python-3.11.12 torch-2.5.0+cu124 CUDA:0 (Tesla T4, 15095MiB)

/root/.cache/torch/hub/WongKinYiu_yolov9_main/models/experimental.py:243: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don'

📥 Please upload your video file...


KeyboardInterrupt: 

In [ ]:
from IPython import get_ipython
from IPython.display import display, Javascript # Import Javascript from IPython.display
# %%
import cv2
import numpy as np
import base64
import PIL.Image
import io
import torch
import smtplib
from email.message import EmailMessage
import pandas as pd
from geopy.distance import geodesic
import time
import matplotlib.pyplot as plt
from google.colab import drive, output

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
print(model)


AutoShape(
  (model): DetectMultiBackend(
    (model): DetectionModel(
      (model): Sequential(
        (0): Silence()
        (1): Conv(
          (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (act): ReLU(inplace=True)
        )
        (2): Conv(
          (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (act): ReLU(inplace=True)
        )
        (3): RepNCSPELAN4(
          (cv1): Conv(
            (conv): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1))
            (act): ReLU(inplace=True)
          )
          (cv2): Sequential(
            (0): RepNCSP(
              (cv1): Conv(
                (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
                (act): ReLU(inplace=True)
              )
              (cv2): Conv(
                (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
                (act): ReLU(inplace=True)
              )
              (cv3): Conv(
         

In [ ]:
#Realtime webcam detection
import cv2
import numpy as np
import base64
import PIL.Image
import io
import torch
import smtplib
from email.message import EmailMessage
import pandas as pd
from geopy.distance import geodesic
import time
import matplotlib.pyplot as plt
from google.colab import drive, output
from IPython.display import Javascript, display

# Mount Google Drive
drive.mount('/content/drive')

# Load YOLOv9 Fire Detection Model
MODEL_PATH = "/content/drive/MyDrive/yolov9/SAVED_BEST.pt"
model = torch.hub.load('WongKinYiu/yolov9', 'custom', path=MODEL_PATH, source='github')

# Load fire stations data
fire_stations_df = pd.read_csv("/content/drive/MyDrive/yolov9/coimbatore_region_fire_stations.csv")
fire_stations_df.rename(columns={"Latitude": "latitude", "Longitude": "longitude"}, inplace=True)

# Fixed Fire Location
fire_latitude, fire_longitude = 10.9993751, 77.0843437
maps_link = f"https://www.google.com/maps?q={fire_latitude},{fire_longitude}"

# Find nearest fire station
def find_nearest_fire_station(fire_lat, fire_lon):
    min_distance = float("inf")
    nearest_station = None
    for _, row in fire_stations_df.iterrows():
        station_coords = (row["latitude"], row["longitude"])
        fire_coords = (fire_lat, fire_lon)
        distance = geodesic(station_coords, fire_coords).kilometers
        if distance < min_distance:
            min_distance = distance
            nearest_station = row
    return nearest_station, min_distance

# Email Credentials
SENDER_EMAIL = "yolov9project@gmail.com"
APP_PASSWORD = "yfus foac dwcu wrgm"

def send_email_alert(recipient_email, subject, body):
    if not recipient_email:
        print("❌ No email address found!")
        return
    msg = EmailMessage()
    msg["Subject"] = subject
    msg["From"] = SENDER_EMAIL
    msg["To"] = recipient_email
    msg.set_content(body)
    try:
        with smtplib.SMTP_SSL("smtp.gmail.com", 465) as server:
            server.login(SENDER_EMAIL, APP_PASSWORD)
            server.send_message(msg)
        print(f"✅ Email alert sent to {recipient_email}!")
    except Exception as e:
        print(f"❌ Failed to send email: {e}")

# JavaScript to capture frames from webcam
js = Javascript('''
    async function streamVideo() {
        const video = document.createElement('video');
        const canvas = document.createElement('canvas');
        const context = canvas.getContext('2d');
        const stream = await navigator.mediaDevices.getUserMedia({ video: true });
        document.body.appendChild(video);
        video.srcObject = stream;
        await new Promise((resolve) => video.onloadedmetadata = resolve);
        video.play();
        canvas.width = video.videoWidth;
        canvas.height = video.videoHeight;
        while (true) {
            context.drawImage(video, 0, 0, canvas.width, canvas.height);
            let imageData = canvas.toDataURL('image/jpeg', 0.8);
            google.colab.kernel.invokeFunction('notebook.captureFrame', [imageData], {});
            await new Promise(resolve => setTimeout(resolve, 100));
        }
    }
    streamVideo();
''')

def base64_to_image(data):
    header, encoded = data.split(',', 1)
    img_bytes = base64.b64decode(encoded)
    img = PIL.Image.open(io.BytesIO(img_bytes))
    return cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)

def capture_frame(image_data):
    global frame
    frame = base64_to_image(image_data)

output.register_callback('notebook.captureFrame', capture_frame)
display(js)

def detect_fire(frame):
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame_resized = cv2.resize(frame_rgb, (640, 640))
    results = model(frame_resized)
    results_df = results.pandas().xywh[0]
    fire_detected = results_df[(results_df['name'] == 'fire') & (results_df['confidence'] >= 0.05)]
    smoke_detected = results_df[(results_df['name'] == 'smoke') & (results_df['confidence'] >= 0.05)]
    return fire_detected, smoke_detected

print("🔥 Fire and Smoke detection started... Press stop to exit.")
frame_counter = 0
frame_interval = 10
while True:
    if 'frame' in globals() and frame is not None:
        frame = frame.copy()
        frame_counter += 1
        if frame_counter % frame_interval == 0:
            print(f"Processing frame {frame_counter}...")
            plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            plt.axis('off')
            plt.show()
            fire_results, smoke_results = detect_fire(frame)
            if not fire_results.empty:
                for _, row in fire_results.iterrows():
                    print(f"🔥 Fire detected! Confidence: {row['confidence']:.2f}")
                nearest_fire_station, distance_to_station = find_nearest_fire_station(fire_latitude, fire_longitude)
                if nearest_fire_station is not None:
                    fire_station_email = nearest_fire_station.get("email", "").strip()
                    email_body = (f"🔥 Fire detected at KIT College!\n\n"
                                  f"📍 Location: {fire_latitude}, {fire_longitude}\n"
                                  f"🌍 View on Map: {maps_link}\n"
                                  f"🚒 Nearest Fire Station: {nearest_fire_station['Station_Name']}\n"
                                  f"📏 Distance: {distance_to_station:.2f} km\n\n"
                                  "Immediate action required!")
                    send_email_alert(fire_station_email, "🔥 Fire Alert!", email_body)
            if not smoke_results.empty:
                for _, row in smoke_results.iterrows():
                    print(f"💨 Smoke detected! Confidence: {row['confidence']:.2f}")
                    email_body = (f"💨 Smoke detected at KIT College!\n\n"
                                  f"📍 Location: {fire_latitude}, {fire_longitude}\n"
                                  f"🌍 View on Map: {maps_link}\n"
                                  "⚠ Please check immediately.")
                    send_email_alert(SENDER_EMAIL, "💨 Smoke Alert!", email_body)
            if fire_results.empty and smoke_results.empty:
                print("✅ No fire or smoke detected.")
        time.sleep(0.5)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Using cache found in /root/.cache/torch/hub/WongKinYiu_yolov9_main
YOLO 🚀 2025-4-24 Python-3.11.12 torch-2.5.0+cu124 CUDA:0 (Tesla T4, 15095MiB)

/root/.cache/torch/hub/WongKinYiu_yolov9_main/models/experimental.py:243: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded

<IPython.core.display.Javascript object>

🔥 Fire and Smoke detection started... Press stop to exit.
Processing frame 10...
💨 Smoke detected! Confidence: 0.41


/root/.cache/torch/hub/WongKinYiu_yolov9_main/models/common.py:1063: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


✅ Email alert sent to yolov9project@gmail.com!
Processing frame 20...
💨 Smoke detected! Confidence: 0.41
✅ Email alert sent to yolov9project@gmail.com!


KeyboardInterrupt: 

In [ ]:
import cv2
import numpy as np
import base64
import PIL.Image
import io
import torch
import smtplib
from email.message import EmailMessage
import pandas as pd
from geopy.distance import geodesic
import time
import matplotlib.pyplot as plt
from google.colab import drive, output
from IPython.display import Javascript, display

# Mount Google Drive
drive.mount('/content/drive')

# Load YOLOv9 Fire Detection Model
MODEL_PATH = "/content/drive/MyDrive/yolov9/SAVED_BEST.pt"
model = torch.hub.load('WongKinYiu/yolov9', 'custom', path=MODEL_PATH, source='github', force_reload=True)

# Load fire stations data
fire_stations_df = pd.read_csv("/content/drive/MyDrive/yolov9/coimbatore_region_fire_stations.csv")
fire_stations_df.rename(columns={"Latitude": "latitude", "Longitude": "longitude"}, inplace=True)

# Fixed Fire Location
fire_latitude, fire_longitude = 10.9993751, 77.0843437
maps_link = f"https://www.google.com/maps?q={fire_latitude},{fire_longitude}"

# Find nearest fire station
def find_nearest_fire_station(fire_lat, fire_lon):
    min_distance = float("inf")
    nearest_station = None
    for _, row in fire_stations_df.iterrows():
        station_coords = (row["latitude"], row["longitude"])
        fire_coords = (fire_lat, fire_lon)
        distance = geodesic(station_coords, fire_coords).kilometers
        if distance < min_distance:
            min_distance = distance
            nearest_station = row
    return nearest_station, min_distance

# Email Credentials
SENDER_EMAIL = "yolov9project@gmail.com"
APP_PASSWORD = "yfus foac dwcu wrgm"

def send_email_alert(recipient_email, subject, body):
    if not recipient_email:
        print("❌ No email address found!")
        return
    msg = EmailMessage()
    msg["Subject"] = subject
    msg["From"] = SENDER_EMAIL
    msg["To"] = recipient_email
    msg.set_content(body)
    try:
        with smtplib.SMTP_SSL("smtp.gmail.com", 465) as server:
            server.login(SENDER_EMAIL, APP_PASSWORD)
            server.send_message(msg)
        print(f"✅ Email alert sent to {recipient_email}!")
    except Exception as e:
        print(f"❌ Failed to send email: {e}")

# JavaScript to capture frames from webcam
js = Javascript('''
    async function streamVideo() {
        const video = document.createElement('video');
        const canvas = document.createElement('canvas');
        const context = canvas.getContext('2d');
        const stream = await navigator.mediaDevices.getUserMedia({ video: true });
        document.body.appendChild(video);
        video.srcObject = stream;
        await new Promise((resolve) => video.onloadedmetadata = resolve);
        video.play();
        canvas.width = video.videoWidth;
        canvas.height = video.videoHeight;
        while (true) {
            context.drawImage(video, 0, 0, canvas.width, canvas.height);
            let imageData = canvas.toDataURL('image/jpeg', 0.8);
            google.colab.kernel.invokeFunction('notebook.captureFrame', [imageData], {});
            await new Promise(resolve => setTimeout(resolve, 100));
        }
    }
    streamVideo();
''')

def base64_to_image(data):
    header, encoded = data.split(',', 1)
    img_bytes = base64.b64decode(encoded)
    img = PIL.Image.open(io.BytesIO(img_bytes))
    return cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)

def capture_frame(image_data):
    global frame
    frame = base64_to_image(image_data)

output.register_callback('notebook.captureFrame', capture_frame)
display(js)

def detect_fire(frame):
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame_resized = cv2.resize(frame_rgb, (640, 640))
    torch.cuda.empty_cache()  # Clears memory cache before inference
    results = model(frame_resized)
    results_df = results.pandas().xywh[0]
    fire_detected = results_df[(results_df['name'] == 'fire') & (results_df['confidence'] >= 0.05)]
    smoke_detected = results_df[(results_df['name'] == 'smoke') & (results_df['confidence'] >= 0.05)]
    return fire_detected, smoke_detected

print("🔥 Fire and Smoke detection started... Press stop to exit.")
frame_counter = 0
frame_interval = 10
while True:
    if 'frame' in globals() and frame is not None:
        frame = frame.copy()
        frame_counter += 1
        if frame_counter % frame_interval == 0:
            print(f"Processing frame {frame_counter}...")
            plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            plt.axis('off')
            plt.show()
            fire_results, smoke_results = detect_fire(frame)
            if not fire_results.empty:
                for _, row in fire_results.iterrows():
                    print(f"🔥 Fire detected! Confidence: {row['confidence']:.2f}")
                nearest_fire_station, distance_to_station = find_nearest_fire_station(fire_latitude, fire_longitude)
                if nearest_fire_station is not None:
                    fire_station_email = nearest_fire_station.get("email", "").strip()
                    email_body = (f"🔥 Fire detected at KIT College!\n\n"
                                  f"📍 Location: {fire_latitude}, {fire_longitude}\n"
                                  f"🌍 View on Map: {maps_link}\n"
                                  f"🚒 Nearest Fire Station: {nearest_fire_station['Station_Name']}\n"
                                  f"📏 Distance: {distance_to_station:.2f} km\n\n"
                                  "Immediate action required!")
                    send_email_alert(fire_station_email, "🔥 Fire Alert!", email_body)
            if not smoke_results.empty:
                for _, row in smoke_results.iterrows():
                    print(f"💨 Smoke detected! Confidence: {row['confidence']:.2f}")
                    send_email_alert(SENDER_EMAIL, "💨 Smoke Alert!", f"💨 Smoke detected at KIT College!\n\n📍 Location: {fire_latitude}, {fire_longitude}\n🌍 View on Map: {maps_link}\n⚠ Please check immediately.")
            if fire_results.empty and smoke_results.empty:
                print("✅ No fire or smoke detected.")
        time.sleep(0.2 )  # Reduced delay for faster frame updates

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Downloading: "https://github.com/WongKinYiu/yolov9/zipball/main" to /root/.cache/torch/hub/main.zip
YOLO 🚀 2025-4-24 Python-3.11.12 torch-2.5.0+cu124 CUDA:0 (Tesla T4, 15095MiB)

/root/.cache/torch/hub/WongKinYiu_yolov9_main/models/experimental.py:243: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don'

<IPython.core.display.Javascript object>

🔥 Fire and Smoke detection started... Press stop to exit.
Processing frame 10...
🔥 Fire detected! Confidence: 0.79
🔥 Fire detected! Confidence: 0.38


/root/.cache/torch/hub/WongKinYiu_yolov9_main/models/common.py:1063: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
